In [35]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohammedalkooheji/guitar-notes-dataset")

print("Path to dataset files:", path)

100%|██████████| 170M/170M [00:15<00:00, 11.2MB/s] 

Extracting files...


Path to dataset files: C:\Users\patma\.cache\kagglehub\datasets\mohammedalkooheji\guitar-notes-dataset\versions\3


In [ ]:
from keras.utils import audio_dataset_from_directory

trainData = audio_dataset_from_directory(
    path+"/Guitar Dataset",
    validation_split = 0.2,
    subset = "training",
    seed = 42
)

validationData = audio_dataset_from_directory(
    path+"/Guitar Dataset",
    validation_split = 0.2,
    subset = "validation",
    seed = 42
)

test_ds = audio_dataset_from_directory(
    path + "/Guitar Dataset",
    shuffle=False
)

Found 1452 files belonging to 43 classes.
Using 1162 files for training.
Found 1452 files belonging to 43 classes.
Using 290 files for validation.
Found 1452 files belonging to 43 classes.


In [40]:
inputs = keras.Input(shape=(32, 88200, 1))
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [37]:
for data_batch, labels_batch in trainData:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

data batch shape: (32, 88200, 1)
labels batch shape: (32,)


In [41]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss",
    )
]
history = model.fit(
    trainData,
    epochs=50,
    validation_data=validationData,
    callbacks=callbacks,
)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node StatefulPartitionedCall/functional_2_1/conv2d_6_1/Relu defined at (most recent call last):
<stack traces unavailable>
Computed output size would be negative: -1 [input_size: 1, effective_filter_size: 3, stride: 1]
	 [[{{node StatefulPartitionedCall/functional_2_1/conv2d_6_1/Relu}}]] [Op:__inference_one_step_on_iterator_2893]

In [ ]:
print("Test")